1\. Write a function that converts number representation (bin<->dec<->hex)

In [17]:
def convert(number=3):
    if type(number)==int:
        return bin(number),hex (number)
    elif type(number) == str :
        if number[1]=='x':
            return int(number,16)
        elif number[1]=='b': 
            return int(number,2)

print(convert(5))
print(convert('0b1110'))
print(convert('0x9D70'))


('0b101', '0x5')
14
40304


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [30]:
def bittosingle(word):
    if word[2]==0:
        sign = -1
    else : 
        sign = 1
    exponent = int('0b' + word[3:11],2) - 127
    mantissabit = word[11:]
    mantissa=0
    for b in mantissabit :
        i=0        
        if b == '1':
            mantissa = mantissa + 1//2**i
        i+=1
    floatingpoint = (1+mantissa)*sign * 2**exponent
    return sign, mantissa, exponent,floatingpoint
    
    
print(bittosingle('0b00000100001101001101011100011010'))


(1, 12, -119, 1.9560225997682864e-35)


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [108]:
import sys
underflow=1
overflow =1

i=1
while underflow!=0: 
    underflow=underflow/2
    i+=1
    
print(sys.getsizeof(underflow),i,underflow)

j=1 
while overflow !=0 :
    overflow=overflow*2
    i+=1
print(sys.getsizeof(overflow),i,overflow)

16 1076 0.0


KeyboardInterrupt: 

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [51]:
acc=4 
e=0
while acc+1.0*10**-e != acc :
    e+=1

print("precision: ",10**(-(e-1)))

precision:  1e-15


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [54]:
import math
def sol(a,b,c):
    return (-b+(b**2-4*a*c)**0.5)/(2*a),(-b-(b**2-4*a*c)**0.5)/(2*a)
#a)
print(sol(0.001,1000,0.001))

#b)
def solbis(a,b,c):
    return (-b+(b**2-4*a*c)**0.5)*(-b+(b**2-4*a*c)**0.5)/(2*a*(-b+(b**2-4*a*c)**0.5)),(-b-(b**2-4*a*c)**0.5)*(-b-(b**2-4*a*c)**0.5)/(2*a*(-b-(b**2-4*a*c)**0.5))

print(solbis(0.001,1000,0.001))

#c)

(-9.999894245993346e-07, -999999.999999)
(-9.999894245993346e-07, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [57]:
def f(x) : return x*(x-1)

#a)
delta = 10**-2
def df (x,d) :
    return (f(x+d)-f(x))/d

print (df(1,delta)) 

#b) 
for e in [4,6,8,10,12,14] : print (df(1,10**-e)) 

1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [103]:
#a)
import numpy as np

def i(N):
    return np.sum([(2/(N+1)) * ((1-k**2)**0.5) for k in range (1,N+1)])

print(i(100))

#b)
import time  

def ibis(t):
    s=0
    t2=0
    k=1
    t1 = time.time() 
    while t2-t1<t :
        s=s+(2/(k+1)) * ((1-k**2)**0.5)
        k+=1
        t2 =time.time() 
    return s,k

#1sec
print(ibis(1))

#1min
#print(ibis(60))

print("diff = ",(9.908642492889024e-11+1618204.1221661477j)-(6.294732297646591e-09+102800779.80403715j))


(6.1194489382275564e-15+99.93818531985147j)
((1.0833200747905921e-10+1769195.9437526872j), 884613)
diff =  (-6.195645872717701e-09-101182575.68187101j)
